In [15]:
import codecs
import numpy as np
import tensorflow as tf
from konlpy.tag import Twitter
from gensim.models import doc2vec

In [ ]:
twitter = Twitter()

In [2]:
model = doc2vec.Doc2Vec.load('../../embed/doc2vec_model/doc2vec_twitter_kowiki_300000_docs.model')

In [80]:
max_sent_len = 30
vocab_size = len(model.wv.vocab)
vector_size = model.vector_size

In [19]:
def tagger(sent):
    return [word for word, tag in twitter.pos(sent) if tag not in ['Punctuation', 'Unknown']]

In [4]:
def vectorize(model):
    def vec(sent):
        return model.wv[filter(lambda x: x in model.wv.vocab, tagger(sent))]
    
    return vec

vec = vectorize(model)

In [70]:
def padding(max_len):
    def pad(sent):
        if len(sent) > max_len:
            return sent[:max_len]
        else:
            return np.vstack((sent, np.zeros((max_len - sent.shape[0], sent.shape[1]))))
    
    return pad

In [127]:
with codecs.open('../sample_data/movie_review/train/train_data', encoding='utf-8') as f:
    data = f.readlines()
    data = map(vec, data)
    data = list(map(padding(max_sent_len), data))

In [78]:
with open('../sample_data/movie_review/train/train_label') as f:
    label = f.readlines()
    label = list(map(int, label))

In [128]:
plc_embed = tf.placeholder(tf.int32, (None, max_sent_len, vector_size))

In [129]:
fw_1 = tf.nn.rnn_cell.LSTMCell(128)
bw_1 = tf.nn.rnn_cell.LSTMCell(128)

In [131]:
bi_lstm_1 = tf.nn.bidirectional_dynamic_rnn(fw_1, bw_1, tf.transpose(plc_embed, (1, 0, 2)), dtype=tf.float32)

ValueError: Initializer for variable bidirectional_rnn/fw/lstm_cell/kernel/ is from inside a control-flow construct, such as a loop or conditional. When creating a variable inside a loop or conditional, use a lambda as the initializer.